# Guía Completa: Lógica Proposicional en Python

## 1. Fundamentos Teóricos

### El Problema
Queremos llenar una tabla 2x2 con números del 0 al 3, sin repetir ningún número.

```
┌───┬───┐
│ ? │ ? │
├───┼───┤
│ ? │ ? │
└───┴───┘
```

### Representación con Lógica Proposicional
Usamos átomos de la forma: `OenCasilla(x, y, n)`
- **x, y**: coordenadas de la casilla (0,0), (0,1), (1,0), (1,1)  
- **n**: número que queremos colocar (0, 1, 2, 3)
- **Significado**: "El número n está en la casilla (x,y)"

## 2. Conectivos Lógicos

### Conjunción (Y) - Itoria ∧
- **Símbolo**: ∧ (en código: "Y")
- **Significado**: "Y además", "ambas cosas deben ser ciertas"
- **Ejemplo**: A ∧ B es verdadero solo si A es verdadero Y B es verdadero

### Disyunción (O) - Otoria ∨  
- **Símbolo**: ∨ (en código: "O")
- **Significado**: "O", "al menos una debe ser cierta"
- **Ejemplo**: A ∨ B es verdadero si A es verdadero O B es verdadero (o ambos)

### Implicación (>)
- **Símbolo**: → (en código: ">")
- **Significado**: "Si... entonces..."
- **Ejemplo**: A → B significa "Si A es verdadero, entonces B debe ser verdadero"

### Negación (-)
- **Símbolo**: ¬ (en código: "-")
- **Significado**: "No es cierto que..."
- **Ejemplo**: ¬A significa "No A"

## 3. Las Tres Restricciones del Problema

### Restricción 1: Un número solo está en una casilla
**Fórmula matemática:**
$$\bigwedge_{x,y} \bigwedge_{n} \left(OenCasilla(x,y,n) \to \neg\left(\bigvee_{(u,v) \neq (x,y)} OenCasilla(u,v,n)\right)\right)$$

**En español**: "Para cada casilla (x,y) y cada número n: Si el número n está en la casilla (x,y), entonces n NO está en ninguna otra casilla"

**Ejemplo específico**: Si el número 2 está en (0,0), entonces el 2 NO está en (0,1) NI en (1,0) NI en (1,1)

### Restricción 2: No más de un número por casilla  
**Fórmula matemática:**
$$\bigwedge_{x,y} \bigwedge_{n} \left(OenCasilla(x,y,n) \to \neg\left(\bigvee_{m \neq n} OenCasilla(x,y,m)\right)\right)$$

**En español**: "Para cada casilla (x,y) y cada número n: Si el número n está en la casilla (x,y), entonces ningún otro número está en esa misma casilla"

**Ejemplo específico**: Si el número 2 está en (0,0), entonces el 0, 1 y 3 NO están en (0,0)

### Restricción 3: Al menos un número por casilla
**Fórmula matemática:**  
$$\bigwedge_{x,y} \left(\bigvee_{n} OenCasilla(x,y,n)\right)$$

**En español**: "Para cada casilla (x,y): debe haber al menos un número en esa casilla"

**Ejemplo específico**: En la casilla (0,0) debe estar el 0 O el 1 O el 2 O el 3

## 4. Cómo Implementar Itorias y Otorias

### Estructura General de una Itoria (∧)
```python
formula_general = ''
primero = True

for elemento in conjunto:
    # Crear subfórmula para este elemento
    subformula = crear_subformula(elemento)
    
    if primero:
        formula_general = subformula
        primero = False
    else:
        formula_general = "(" + formula_general + "Y" + subformula + ")"
```

### Estructura General de una Otoria (∨)
```python
formula_or = ''
inicial_or = True

for elemento in conjunto:
    atomo = crear_atomo(elemento)
    
    if inicial_or:
        formula_or = atomo
        inicial_or = False
    else:
        formula_or = "(" + formula_or + "O" + atomo + ")"
```

## 5. Implementación Paso a Paso

### Paso 1: Configuración Inicial
```python
from Logica import Descriptor, visualizar_formula

Nx, Ny = 2, 2  # Dimensiones de la tabla
Nn = Nx * Ny   # Cantidad de números (4)
X = list(range(Nx))      # [0, 1]
Y = list(range(Ny))      # [0, 1]  
numeros = list(range(Nn)) # [0, 1, 2, 3]

# Crear descriptor para codificar OenCasilla(x,y,n)
OenCasilla = Descriptor([Nx, Ny, Nn])
```

### Paso 2: Restricción 1 - Un número en una sola casilla
```python
def restriccion_1():
    formula_general = ''
    primero = True
    
    # Para cada casilla (x,y)
    for x in X:
        for y in Y:
            # Para cada número n
            for n in numeros:
                # Encontrar todas las otras casillas
                otras_casillas = [(i,j) for i in X for j in Y if (i,j) != (x,y)]
                
                # Crear la otoria: "n está en otra casilla"
                formula_or = ''
                inicial_or = True
                for u, v in otras_casillas:
                    atomo = OenCasilla.ravel([u, v, n])
                    if inicial_or:
                        formula_or = atomo
                        inicial_or = False
                    else:
                        formula_or = "(" + formula_or + "O" + atomo + ")"
                
                # Crear implicación: Si n está en (x,y) entonces NO está en otras
                antecedente = OenCasilla.ravel([x, y, n])
                consecuente = "-" + formula_or if formula_or else ""
                subformula = "(" + antecedente + ">" + consecuente + ")"
                
                # Agregar a la itoria general
                if primero:
                    formula_general = subformula
                    primero = False
                else:
                    formula_general = "(" + formula_general + "Y" + subformula + ")"
    
    return formula_general
```

### Paso 3: Restricción 2 - No más de un número por casilla
```python
def restriccion_2():
    formula_general = ''
    primero = True
    
    # Para cada casilla (x,y)
    for x in X:
        for y in Y:
            # Para cada número n
            for n in numeros:
                # Encontrar todos los otros números
                otros_numeros = [m for m in numeros if m != n]
                
                # Crear la otoria: "otro número está en (x,y)"
                formula_or = ''
                inicial_or = True
                for m in otros_numeros:
                    atomo = OenCasilla.ravel([x, y, m])
                    if inicial_or:
                        formula_or = atomo
                        inicial_or = False
                    else:
                        formula_or = "(" + formula_or + "O" + atomo + ")"
                
                # Crear implicación: Si n está en (x,y) entonces otros NO están
                antecedente = OenCasilla.ravel([x, y, n])
                consecuente = "-" + formula_or if formula_or else ""
                subformula = "(" + antecedente + ">" + consecuente + ")"
                
                # Agregar a la itoria general
                if primero:
                    formula_general = subformula  
                    primero = False
                else:
                    formula_general = "(" + formula_general + "Y" + subformula + ")"
    
    return formula_general
```

### Paso 4: Restricción 3 - Al menos un número por casilla
```python
def restriccion_3():
    formula_general = ''
    primero = True
    
    # Para cada casilla (x,y)
    for x in X:
        for y in Y:
            # Crear otoria: "algún número está en (x,y)"
            formula_or = ''
            inicial_or = True
            
            for n in numeros:
                atomo = OenCasilla.ravel([x, y, n])
                if inicial_or:
                    formula_or = atomo
                    inicial_or = False
                else:
                    formula_or = "(" + formula_or + "O" + atomo + ")"
            
            # Agregar esta otoria a la itoria general
            if primero:
                formula_general = formula_or
                primero = False
            else:
                formula_general = "(" + formula_general + "Y" + formula_or + ")"
    
    return formula_general
```

## 6. Patrones Clave para Recordar

### Patrón de Itoria Anidada (∧∧)
```python
# ∧x ∧y ∧n Formula(x,y,n)
for x in X:
    for y in Y:
        for n in numeros:
            # Construir subfórmula
            # Agregar con "Y" a fórmula general
```

### Patrón de Otoria Interna (∨ dentro de ∧)  
```python
# ∧x (∨n Formula(x,n))
for x in X:
    # Construir otoria interna
    formula_or = ''
    for n in numeros:
        # Agregar con "O"
    # Agregar otoria completa con "Y" a fórmula general
```

### Patrón de Implicación con Negación
```python
# P → ¬(∨ Q)
antecedente = "P"
# Construir otoria Q
consecuente = "-(" + otoria_Q + ")"
implicacion = "(" + antecedente + ">" + consecuente + ")"
```

## 7. Consejos de Depuración

1. **Usa visualizar_formula()** para verificar que tu fórmula tiene sentido
2. **Construye paso a paso**: implementa primero casos específicos, luego generaliza
3. **Verifica los paréntesis**: cada operador binario necesita paréntesis
4. **Variables iniciales**: siempre usa banderas como `primero = True` para el primer elemento
5. **Prueba con ejemplos pequeños**: antes de hacer el caso general

La clave está en entender que estás traduciendo lógica matemática formal a código Python, manteniendo la estructura de las conjunciones y disyunciones anidadas.